# Evaluating the IMPALA Model Fit
In this document, we introduce some IMPALA functionality for evaluating the quality of the IMPALA calibration in terms of MCMC diagnostics/convergence, prediction in the training data, and statistical coverage of corresponding uncertainties. This is a non-exhaustive set of analyses that one can use to diagnose IMPALA outputs, but it should provide a solid foundational toolkit. We refer interested users to our other tutories (e.g., troubleshooting_convergence) for additional details on specific issues.


## 1. Diagnosing the MCMC Sampler
IMPALA uses Markov Chain Monte Carlo sampling for parameter estimation, and users should do at minimum some basic evaluation of the performance of the sampler. Additional details are provided elsewhere; here, we just introduce some default functionality available to users with some basic visualization. 

### Trace Plots
Trace plots show the posterior draws for various parameters (y-axis) as a function of the number of iterations of the MCMC sampler (x-axis). Visual evaluation of these plots provides a sense of how well the sampler has converged, how correlated parameter draws are across iterations and between different parameters, how many iterations to toss out at the start of the MCMC chain as burn-in, etc. 

Here is an example of code one might run to plot to theta0 draws for a hierarchical calibration stored in *out*:

    import impala.superCal.post_process as pp
    pp.parameter_trace_plot(out.theta0[:,0,:], ylim=[0, 1])

Here is an example trace plot. Notice the high autocorrelation evident in the last two parameters. In this calibration, the last two parameters were poorly identified with essentially uniform posteriors.
![something](./images/trace_example.png)

### Performance of Parallel Tempering
IMPALA uses a sophisticated parameter sampling method called parallel tempering, which allows the sampler to more easily move between local modes of the posterior distribution. Parallel tempering works by running many parallel MCMC chains with different properties and at each iteration proposing swaps between chains. One useful diagnostic of the health of the parallel tempering is the proportion of iterations in which a proposed swap between parallel chains is accepted. Ideally, we would like this acceptance rate somewhere between 20-60%. 

For pooled and hierarchical calibrations, a diagnostic plot can be generated for a calibration stored in *out* and a IMPALA setup object stored in *setup*:

    pp.total_temperature_swaps(out,setup)

Here is an example output plot. The total proportion of swaps accepted is shown in the text, and the proportion of swaps between pairs of chains (indexed by temperatures) is shown in a heatmap. This calibration shows some evidence of sub-optimal tempering performance. 

![something](./images/tempering_example.png)

### Additional Convergence Diagnostics
Trace plots and proportion of parallel swaps accepted are the minimal set of convergence diagnostics one might want to run. However, many more diagnostics exist. The pyMC module in Python provides some useful functions for evaluating convergence (e.g., https://pymcmc.readthedocs.io/en/latest/modelchecking.html), and these methods can be applied to many objects stored in the outputted IMPALA calibration object to evaluate convergence.

## 2. Plotting the Posterior Draws for Theta

### Pairs Plots
Pairs plots show parameter draws for pairs of parameters. A standard pairs plot can be generated using the *pairs* function, and specialized pairs plots for each calibration type can also be generated. 

    pp.pairs(...)
    pp.pairwise_theta_plot_pool(...)
    pp.pairwise_theta_plot_hier(...)
    pp.pairwise_theta_plot_cluster(...)

Here is an example of a plot generated by *pairwise_theta_plot_hier*. Parameter theta_i indicates experiment-specific parameter draws, theta_0 indicates the mean of the parent distribution for theta_i's, and theta_star indicates the parent distribution itself (i.e., theta_0 draws + additional noise). 

![something](./images/pairs_hier_example.png)

### Experiment-Specific Parameter Boxplots (Hierarchical and Clustering Only)

Suppose we have a setup object with a single *addVecExperiments* call and that nexp is the number of unique theta_i value/experiments entered by the single call. Here is some code that can be used to generate boxplots of the theta_i values across experiments. This type of plot is useful for identifying how variable the theta_i values are across experiments. 

    import numpy as np
    import pandas as pd
    import seaborn as sns
    KEYS = np.array(pd.DataFrame(setup.bounds.keys())).flatten()
    theta_i = []
    for j in range(nexp):
        mat = pd.DataFrame(np.array(pd.DataFrame(sc.tran_unif(out.theta[0][:,0,j,:],setup.bounds_mat, setup.bounds.keys()).values())).T, columns = KEYS)
        mat['exp'] = j
        if j == 0:
            theta_i = [mat]
        else:
            theta_i.append(mat)
    theta_i_long = pd.concat(theta_i)

    fig,ax=plt.subplots(1,setup.p,figsize=(20,4))
    fig.tight_layout(pad=2.0)
    for j in range(setup.p):
        ax[j].set_xlabel('Experiment')
        ax[j].set_ylabel('')
        ax[j].set_title(KEYS[j]) 
        A = sns.boxplot(data = theta_i_long, x = theta_i_long['exp'], y = theta_i_long[KEYS[j]], ax=ax[j],showfliers=False)
        A.set(xlabel='Experiment',ylabel='')
        A.set(xticklabels=[]) 

Here is an example output: 
![something](./images/thetai_example.png)



## 3. Evaluating Predictions in Training Data
Another important diagnostic is how well the calibrated model predicts the training data. Poor predictions can indicate model discrepancy, errors in implementation, etc. IMPALA provides several utilities for calculating and visualizing predictions. 

### Getting and Plotting Predictions
The function *get_outcome_predictions_impala* can be used to generate predictions for all training data points and a user-specified set of theta values. The *post_process.py* script also has various other functions that help with plotting these predictions. 

For example, prediction plots specific to ModelMaterialStrength calibrations can be generated as follows:

    pp.ptw_prediction_plots_pool(...)
    pp.ptw_prediction_plots_hier(...)
    pp.ptw_prediction_plots_cluster(...)

Generate prediction plots can be generated using the *func_prediction_plot* function. 

### Cross-Experiment Prediction Errors (Hierarchical and Clustering Only)

One extremely useful diagnostic for identifying outliers and determining if there are subsets of experiments that behave "differently" than others is to calculate the predictions for experiment i using the experiment-specific theta values estimated for other experiments. If an experiment has low prediction error using its own theta_i estimate but high prediction error using other experiments' theta estimates, there is some evidence that the experiment is somehow "different" than the others. 

Here, we provide some code for generating these prediction errors in a particular setting. This code can be adapted by users to accomodate different model structures. For this demonstration, we suppose we have a setup object with a single *addVecExperiments* call and that nexp is the number of unique theta_i value/experiments entered by the single call. 

    THETAi_Y = [get_outcom_predictions_impala(setup, theta_input = np.array(pd.DataFrame(sc.tran_unif(out.theta[0][:,0,j,:],setup.bounds_mat, setup.bounds.keys()).values())).T)['outcome_draws'] for j in range(nexp)]
    CROSS_ERRORS = np.empty([nexp,nexp])
    for i in range(nexp): 
        for j in range(nexp):
            CROSS_ERRORS[i,j] = np.mean(np.abs(setup.ys[0][np.where(setup.s2_ind[0] == j)[0]] -np.mean(THETAi_Y[i][0][:,np.where(setup.s2_ind[0] == j)[0]],axis=0))/setup.ys[0][np.where(setup.s2_ind[0] == j)[0]])
    ax = sns.heatmap(CROSS_ERRORS.T*100, linewidths =0)
    ax.set_xlabel('Theta Index')
    ax.set_ylabel('Prediction Index')
    ax.set_title('Prediction Errors (%)')

Here, we provide an example output. Rows correspond to experiments being predicted and columns correspond to experiment-specific theta_i values. The color in each cell corresponds to the mean absolute percent error (MAPE) for predicting the training data. This figure indicates some differing behavior between the first 7 and last 9 experiments. It also highlights experiment 6 as particularly poorly-predicted using the theta_i values obtained for experiments 7-15. 

![something](./images/cross_errors_example.png)
